In [1]:
#Importing all the important packages 

#For accessing and fetching info from the subtitle database file. Provides an SQL interface.
import sqlite3
#For creation of dataframe and saving subtitle info as csv for future retrieval.
import pandas as pd
#For unzipping the zipped content.
import zipfile
#For working with bytes file.
import io
#For working with paths/directories.
import os

In [3]:
#Path
db_path = "data-20240401T135418Z-001/data/eng_subtitles_database.db"

In [4]:
#Making a connection with the subtitle database file using sqlite3
con = sqlite3.connect(db_path)

In [5]:
#converts byte strings returned by SQLite into python string object
con.text_factory=str

In [96]:
#Retrieving subtitle information 
subtitle_info = con.execute('SELECT * from zipfiles')
sub_info = subtitle_info.fetchall()

In [71]:
#Retrieving subtitle separately content 
content = con.execute('SELECT content from zipfiles')
list_content = content.fetchall()

In [109]:
#Closing connection since we have the data we need now
con.close()

In [73]:
#Decompress files and dump them into the following directory
target_directory = "data-20240401T135418Z-001/static/"

os.makedirs(target_directory, exist_ok=True)

def sanitize_file_name(file_name):
    return ''.join(c if c.isalnum() or c in ['_', '.'] else '_' for c in file_name)

for subtitle in list_content:
    sub = io.BytesIO(subtitle[0])
    with zipfile.ZipFile(sub, 'r') as zip_ref:
        # Extract all files in the ZIP archive
        for file_name in zip_ref.namelist():
            # Remove problematic characters from the file name
            new_file_name = file_name.replace('\t', '_')  # Replace '\t' with '_' or any other suitable replacement
            new_file_name = sanitize_file_name(new_file_name)
            # Construct the full path for the extracted file
            extracted_file_path = os.path.join(target_directory, new_file_name)
            # Extract the file
            with zip_ref.open(file_name) as file_in_zip, open(extracted_file_path, 'wb') as extracted_file:
                extracted_file.write(file_in_zip.read())

In [98]:
#To make a CSV file of id, name, and the file name.
ids=[]
name=[]
mani_name = []
def sanitize_file_name(file_name):
    return ''.join(c if c.isalnum() or c in ['_', '.'] else '_' for c in file_name)
for one_sub in sub_info:
    sub = io.BytesIO(one_sub[2])
    with zipfile.ZipFile(sub, 'r') as zip_ref:
        # Extract all files in the ZIP archive
        for file_name in zip_ref.namelist():
            # Remove problematic characters from the file name
            new_file_name = file_name.replace('\t', '_')  # Replace '\t' with '_' or any other suitable replacement
            new_file_name = sanitize_file_name(new_file_name)
            if '.srt' in new_file_name:
                mani_name.append(new_file_name)
                ids.append(one_sub[0])
                name.append(one_sub[1])

In [104]:
#Checking the consistency in length of lists
print('Length of list of ids:', len(ids))
print('Length of list of name:', len(name))
print('Length of list of mani_name:', len(mani_name))

Length of list of ids: 76038
Length of list of name: 76038
Length of list of mani_name: 76038


In [105]:
#Building a dataframe
sub_dict = {'ids': ids, 'name':name, 'content_name': mani_name}
df = pd.DataFrame(sub_dict)

In [106]:
df.head()

,ids,name,content_name
0,9180533,the.message.(1976).eng.1cd,The.Message.1976.REMASTERED.1080p.BluRay.x264_...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,Here_Comes_the_Grump___Ep._9___Joltin__Jack_in...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,Yumi_s.Cells.S02E13.1080p.WEB_DL.AAC.H.264_Tae...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,Yumi_s.Cells.S02E14.1080p.WEB_DL.AAC.H.264_Tae...
4,9180600,broker.(2022).eng.1cd,Broker_2022_720p_Korean_HDRip_H265_BONE.srt


In [107]:
#Save as csv file
df.to_csv('subtitle_connection.csv', index = False)